# TP ARK avec python

## Le contexte...
Pour obtenir une version dynamique et testable de ce notebook, rendez-vous sur https://mybinder.org/v2/gh/BertrandCaron/ark-tools/master?filepath=TP_ARK_python.ipynb. 

Depuis le début du confinement, j'ai décidé de remettre le nez dans Python, langage sur lequel nous avions eu une formation dispensée par Logilab il y a quelques années. Je n'avais pas pratiqué et j'ai pensé avoir un peu de temps pour m'y consacrer. En réalité, je l'ai fait essentiellement sur mon temps de loisir...

Influencé par un collègue qui m'avait confié ne plus vouloir apprendre des langages liés à un formalisme spécifique, j'avais pour objectif d'apprendre à traiter des données qui n'étaient pas dans des formats dont j'avais l'habitude - XML et RDF, pour ne pas les nommer.

Je cherchais donc un sujet sur lequel essayer mes toutes fraîches compétences. Et l'identifiant ARK s'est vite présenté comme une évidence ! Je vous propose donc ce notebook Jupyter
* pour les bibliothécaires, BnF ou non, qui apprennent python et veulent s'essayer à un petit TP sur un sujet maison (niveau "grand débutant", avoir suivi jusqu'au bout les deux premières ressources ci-dessous devrait suffire à s'en sortir avec brio),
* pour les personnes qui s'intéressent à la génération de noms ARK - celles-ci pourront se dispenser de regarder le code, mais pourraient être intéressées par le texte qui l'entoure -,
* comme une "preuve de concept" sur l'intérêt d'utiliser les Jupyter notebooks pour partager nos expériences de bibliothécaires apprentis ~~sorciers~~ codeurs.

## Ressources consultées

* Library Carpentry, _Python for libraries_. Disponible en ligne sur https://librarycarpentry.org/lc-python-intro/.
* Benjamin Marlé, Alexis Perrier, _Initiez-vous à Python pour l'analyse de données_, OpenClassrooms, mise à jour du 27/04/2020. Disponible en ligne sur https://openclassrooms.com/fr/courses/6204541-initiez-vous-a-python-pour-lanalyse-de-donnees. 
* Jason R. Briggs, _Python for Kids: A Playful Introduction to Programming_, No Starch Press : San Francisco, 2013.
* Vincent Le Goff, _Apprenez à programmer en Python_, OpenClassrooms, mise à jour 7/02/2020. Disponible en ligne sur https://openclassrooms.com/fr/courses/235344-apprenez-a-programmer-en-python/232431-utilisez-des-fichiers. 

## Ressources encore à découvrir...

* Ali Neishabouri, _Découvrez les librairies Python pour la Data Science_, OpenClassrooms, mise à jour du 31/03/2020. Disponible en ligne sur https://openclassrooms.com/fr/courses/4452741-decouvrez-les-librairies-python-pour-la-data-science.

## Petit rappel sur ARK

Vous trouverez de l'information sur le sujet de l'identifiant ARK dans les documents suivants :
* John Kunze (éd.), Bertrand Caron (trad. en français), _Foire aux questions sur l'identifiant ARK_, Projet ARKs-In-The-Open, version du 13 mars 2020. Disponible en ligne sur https://wiki.lyrasis.org/pages/viewpage.action?pageId=178880619.
* Bibliothèque nationale de France, _L’identifiant ARK (Archival Resource Key)_. Disponible en ligne sur https://www.bnf.fr/fr/lidentifiant-ark-archival-resource-key.

Sur la politique de la BnF concernant ARK, on trouvera aussi des informations dans le document suivant :
* Bibliothèque nationale de France, _Préconisations pour l'implémentation d'ARK par les sous-autorités nommantes et autorités d'adressage BnF_, version 2 du 7 mars 2019, identifiant : BnF-ADM-2018-022324-02. Disponible en ligne sur https://multimedia-ext.bnf.fr/pdf/ark_preconisations_bnf.pdf.

Si vous êtes utilisateur de données BnF, vous savez peut-être que la partie spécifique à un document de l'identifiant ARK se termine par un caractère de contrôle. Beaucoup d'identifiants ont fait également ce choix : c'est la clé sur deux caractères de votre numéro de sécurité sociale, c'est le dernier caractère de l'ISBN, etc. Le caractère est produit par l'analyse d'un algorithme assez simple qui analyse les caractères "significatifs" et produit une "clé". Le but étant que, si l'on recopie à la main, ou si par un copier-coller hasardeux on tronque l'identifiant, un recalcul puisse nous avertir que notre copie est fautive.

Les ARK BnF ont tous un caractère de contrôle (en réalité PRESQUE tous, mais on ne va pas s'appesantir sur ce sujet sinon je vais me mettre en boule). Ainsi, dans l'ARK ark:/12148/cb32752361d, le caractère "d" n'est pas significatif mais résulte simplement d'un calcul sur les autres caractères du nom ARK. Si vous cherchez l'ARK [ark:/12148/cb32752361s](https://catalogue.bnf.fr/ark:/12148/cb32752361s) dans notre catalogue, il pourra vous dire que votre ARK est mal formé, plutôt que de sortir une bête erreur 404 comme quoi il n'a pas trouvé ce que vous demandez.

_Note : le nom ARK, c'est la partie spécifique à un objet. Sachant que "ark:" est le libellé de l'identifiant, "12148" est l'identifiant de la BnF dans le registre ARK, donc "cb32752361d" est ici le nom ARK._

## Le problème

Il se trouve que, exilé des outils BnF, j'avais besoin de créer des identifiants exemples qui aient l'air réaliste - donc qui comprennent un caractère de contrôle qui corresponde à la chaîne de caractères qui précède.

Ni une ni deux, je me suis dit que j'allais coder cette affaire-là.

La façon dont nous calculons ce caractère n'est pas un secret, elle est même tout à fait publique puisqu'elle a été piquée à la _California Digital Library_ qui la dévoile ici : https://metacpan.org/pod/distribution/Noid/noid#NOID-CHECK-DIGIT-ALGORITHM.

Je vais vous en faire un petit résumé en français : afin qu'ils soient le plus possible opaques, nos ARK ne contiennent que les caractères suivants : des chiffres, et des consonnes, en évitant la lettre "l" qui peut être confondue avec le chiffre 1 ou un "i" majuscule. Nous avons donc un alphabet de 29 caractères : 0123456789bcdfghjkmnpqrstvwxz.

Voici donc le calcul : on va prendre un nom ARK sans son caractère de contrôle, puis on analyse successivement chacun de ses caractères en multipliant son indice dans la chaîne de caractères de l'alphabet par son indice dans la chaîne de caractères du nom ARK. On additionne le résultat pour chaque position, puis on fait un modulo 29 du résultat, et on déduit de ce nombre (entre 0 et 28, si vous avez suivi) le caractère de contrôle.

Serait-ce peu clair ? On va prendre un exemple ! Mettons que nous ayons la chaîne de caractères 'cb32752361' et que nous voulions en déduire le caractère de contrôle pour connaître son nom ARK. Chaque caractère de l'alphabet a une valeur si on la convertit en base 10, selon le tableau suivant :

| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | b  | c  | d  | f  | g  | h  | j  | k  | m  | n  | p  | q  | r  | s  | t  | v  | w  | x  | z  |
|---|---|---|---|---|---|---|---|---|---|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|
| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 | 24 | 25 | 26 | 27 | 28 |

Ainsi, 'b' a pour valeur 10, 'c' 11, etc.

Pour calculer le caractère de contrôle de la chaîne 'cb32752361', on calculera donc
c (ou plutôt sa valeur décimale, donc 10) x 1 (sa position dans la chaîne de caractères ARK) + b (11) x 2 + 3 x 3 + 2 x 4 + ...

Le total de cette série est 215. Si on divise 215 par 29 et qu'on prend le reste (le principe du modulo), on obtient 12. Et 12, dans le tableau ci-dessus, correspond à 'd', qui est notre caractère de contrôle.

Donc ma proposition est de créer un petit programme python qui va, à partir d'une chaîne de caractères conforme à l'alphabet ARK BnF, renvoyer le bon caractère de contrôle. J'ai pu le coder, il n'y a pas de raison que vous ne puissiez en faire de même ! 

## Une solution

Je vous propose la chose suivante, ça a l'air de marcher, mais je suis sûr que vous pouvez trouver autre chose, mieux, voire des erreurs dans mon code. N'hésitez pas à me faire signe...

Première idée : je vais créer un fichier ARK.py. Parce que pour le moment l'ambition est très limitée, mais on pourrait essayer d'aller plus loin ensuite.

Au tout début, je vais déclarer une variable 'alphabet' pour stocker toutes les lettres autorisées, dans le bon ordre bien sûr.

In [27]:
alphabet ='0123456789bcdfghjkmnpqrstvwxz'

Je commence par créer une fonction simple qui, quand on lui donne un caractère de l'alphabet ARK, renvoie son indice dans l'alphabet. On pourra s'en servir dans plusieurs situations comme ça (ligne 1).

Je vérifie par une structure conditionnelle que le paramètre qu'on a passé à la fonction n'a qu'un seul caractère et que ce caractère figure bien dans l'alphabet (ligne 2).

Je lance un itérateur et une boucle while qui va parcourir les caractères de l'alphabet. Je n'utilise pas la boucle 'for' parce que j'ai besoin de l'indice de chaque caractère, qui m'est fourni par l'itérateur (lignes 3 et 4).

Lorsque la boucle tombe sur l'élément de l'alphabet qui correspond à notre caractère (ligne 5)...

...la fonction renvoie l'indice et s'arrête (ligne 6).

Après avoir créé une boucle infinie comme le noob que je suis, je rajoute la ligne suivante pour incrémenter mon itérateur (ligne 7).

Enfin, si le paramètre passé à la fonction n'est pas un unique caractère ou s'il ne figure pas dans l'alphabet, on aboie sur l'utilisateur de manière précise.

In [28]:
def indiceDansAlphabetARKBnF(caracArkBnf):
    if len(caracArkBnf) == 1 and caracArkBnf in alphabet:
        i = 0
        while i < len(alphabet):
            if alphabet[i] == caracArkBnf:
                    return i
            i += 1
    else:
        print('Cela n\'est pas un caractère de l\'alaphabet ARK BnF valide.')

Réfléchissons. Je créerais bien une bête fonction qui prenne en seul argument la chaîne de caractères mais on m'a un peu appris à programmer orienté objet. Sur quel objet vais-je travailler ? Sur une chaîne de caractères, pour le moment (ligne 1).

Pour chaque chaîne de caractères, je compte ainsi créer une instance de 'chaineArkBnf'. Et je vais lui passer comme propriété de base ma chaîne de caractères. Sauf que je vais aussi lui donner une valeur vide par défaut (mon idée est que ça pourrait être intéressant ensuite de générer une chaîne de caractères aléatoire pour constituer un nom ARK. Dans ce cas-là, je créerais une classe avec une chaîne vide, et je créerais une méthode 'generer' pour obtenir une chaîne aléatoire. Mais n'anticipons pas.) (ligne 2)

J'ai donc créé la méthode nommée '__init__' qui est lancée automatiquement quand on crée une instance de la classe. Dedans, je vais simplement déclarer que la variable 'valeur' a pour valeur la chaîne de caractères qu'on m'a passée en argument (ligne 3).

Maintenant, je vais créer une méthode (une fonction applicable à une instance de classe) qui me permette de valider le fait que tous les caractères de ma chaîne sont bien dans l'alphabet ARK BnF (ligne 5).

Je commence par initialiser une liste vide, où je compte stocker le résultat de l'analyse de chaque caractère de la chaine (ligne 6).

Je parcours ensuite tous les caractères de ma chaîne avec une boucle 'for' (ligne 7).

Et j'introduis une structure conditionnelle : si le caractère ne figure pas dans l'alphabet... (ligne 8)

...eh bien je l'ajoute à ma liste de caractères invalides (ligne 9).

Maintenant que j'ai une liste qui contient éventuellement des caractères invalides, je déclare une condition : si ma liste est vide (si sa longueur égale zéro)... (ligne 10)

...je renvoie un tuple (une liste dont les valeurs ne peuvent être changées) avec comme première valeur le booléen 'True' et en seconde valeur un libellé pour l'humain (ligne 11).

Sinon (si ma liste comprend au moins un élément)... (lgne 12)

... je renvoie un tuple avec en première valeur le booléen 'False', et en seconde valeur une chaîne de caractères où j'intercale ma liste, mais en la convertissant d'abord en 'set' (un set, c'est comme une liste, sauf que ça ne comprend que des valeurs uniques - ça équivaut donc à éliminer les doublons d'une liste) puis en chaîne de caractères (ligne 13).

Pas trop mal pour un début. Maintenant, je vais créer une seconde méthode pour calculer le caractère de contrôle (ligne 15).

J'appelle ma première méthode pour vérifier que tous les caractères sont dans l'alphabet ARK BnF. Pas la peine d'aller plus loin si ce n'est pas le cas (ligne 16).

Je mets une condition pour vérifier que le résultat de mon appel à la méthode 'valider' est bien (qu'en première valeur, on a bien 'True') (ligne 17).

J'initialise une variable qui me permettra de stocker le résultat de mon calcul (vous vous souvenez, l'addition de toutes les valeurs des caractères multipliées par leur indice dans la chaîne) (ligne 18).

Je lance un itérateur pour utiliser une boucle 'while', pour la même raison que dans la fonction qui calcule la position dans l'alphabet ARK. (lignes 19 et 20).

Et j'additionne la multiplication de l'indice du caractère dans l'alphabet (grâce à un appel à ma petite fonction définie au début) par l'indice du caractère dans la chaîne. Je suis obligé d'ajouter 1 parce que le premier indice dans une liste ou une chaîne de caractères pour python, c'est 0, or la valeur du premier caractère de notre chaîne, on la multiplie par 1, pas par 0. J'aurais pu démarrer mon itérateur à 1, mais dans ce cas il aurait fallu que je lui enlève 1 pour obtenir l'indice dans l'alphabet ARK (ligne 21).

Cette fois-ci je ne me laisse pas avoir et je n'oublie pas d'incrémenter l'itérateur (ligne 22).

Je calcule mon caractère de contrôle en faisant le modulo 29 de mon addition puis en prenant dans l'alphabet le caractère qui a cet indice (ligne 23).

Et je le renvoie (ligne 24).

Et si la chaîne n'est pas valide, je renvoie plutôt le second élément du tuple renvoyé par la fonction 'valider', celle compréhensible par l'humain (lignes 25 et 26).

In [29]:
class chaineArkBnf():
    def __init__(self, valeur=''):
        self.valeur = valeur
        
    def valider(self):
        caracInvalides = []
        for carac in self.valeur:
            if carac not in alphabet:
                caracInvalides.append(carac)
        if len(caracInvalides) == 0:
            return (True, 'La chaîne est valide au regard de l\'alphabet BnF.')
        else:
            return (False, 'La chaîne n\'est pas valide, les caractères %s ne figurent pas dans l\'alphabet BnF.' % str(set(caracInvalides)))

    def calculCaracControle(self):
        valide = self.valider()
        if valide[0] == True:
            addition = 0
            i = 0
            while i < len(self.valeur):
                addition += indiceDansAlphabetARKBnF(self.valeur[i]) * (i + 1)
                i += 1
            caracControle = alphabet[addition % 29]
            return caracControle
        else:
            return valide[1]

Et voilà ! Il suffit maintenant de créer une instance de classe 'chaineArkBnf' en lui donnant une chaîne de caractères comme unique paramètre, et ensuite d'appeler sur cette instance la méthode calculCaracControle.

In [30]:
chaine1 = chaineArkBnf('cb32752361')
print(chaine1.calculCaracControle())

d


_Tout ça pour ça ???_

Eh oui. Une minuscule lettre. Mais tout ça va nous permettre de créer des noms ARK complets, de les valider... dans un prochain TP.

Vous trouverez le fichier python produit dans le même dossier, sous le nom ARK-1.py.